In [3]:
import json

# Input and output file paths
input_file = "hotpotqa_similarity_annotations.jsonl"
output_file = "filtered_hotpotqa.jsonl"

def find_first_unsupported_rows(input_file, output_file):
    # Initialize an empty list to store filtered rows
    filtered_rows = []

    # Read the JSONL file
    with open(input_file, 'r') as infile:
        data = json.load(infile)["data"]  # Load the data key from the JSON file

        # Loop through each row in the data
        for row in data:
            # Check the annotation of the first claim
            if len(row["claims"]) > 0 and row["claims"][0]["annotation"] != "S":
                filtered_rows.append(row)

    # Write the filtered rows to a new JSONL file
    with open(output_file, 'w') as outfile:
        json.dump({"data": filtered_rows}, outfile, indent=2)

find_first_unsupported_rows(input_file, output_file)



In [8]:
#Add answer to filtered file
filtered_file = "filtered_hotpotqa.jsonl"
answer_file = "../index_store/conformalhotpot500/hotpot_calibrate_score_500.jsonl"

def add_answer_to_filtered_file(filtered_file, answer_file):
    # Load the filtered data
    with open(filtered_file, 'r') as infile:
        filtered_data = json.load(infile)["data"]

    # Load the answer data
    with open(answer_file, 'r') as infile:
        answer_data = [json.loads(line) for line in infile.readlines()]

    # Create a dictionary mapping question IDs to answers
    answer_dict = {row["query"]: row["answer"] for row in answer_data}

    # Add the answer to each row in the filtered data
    for row in filtered_data:
        row_ordered = {
            "prompt": row["prompt"],
            "answer": answer_dict.get(row["prompt"], "No answer"),
            "original-output": row["original-output"],
            "claims": row["claims"],
        }
        # Update the row in the list
        row.clear()
        row.update(row_ordered)

    # Write the updated data to a new JSONL file
    with open(filtered_file, 'w') as outfile:
        json.dump({"data": filtered_data}, outfile, indent=2)

add_answer_to_filtered_file(filtered_file, answer_file)